<a href="https://colab.research.google.com/github/CodeHunterOfficial/ABC_DataMining/blob/main/NM/%D0%90%D0%BD%D0%B0%D0%BB%D0%B8%D0%B7_%D0%B8_%D0%BF%D1%80%D0%B8%D0%BC%D0%B5%D0%BD%D0%B5%D0%BD%D0%B8%D0%B5_%D0%B4%D0%B8%D1%84%D1%84%D1%83%D0%B7%D0%B8%D0%BE%D0%BD%D0%BD%D1%8B%D1%85_%D0%BC%D0%BE%D0%B4%D0%B5%D0%BB%D0%B5%D0%B9_%D0%BD%D0%B0_%D0%BE%D1%81%D0%BD%D0%BE%D0%B2%D0%B5_Vision_Transformer_%D0%B4%D0%BB%D1%8F_%D0%B7%D0%B0%D0%B4%D0%B0%D1%87_%D0%B3%D0%B5%D0%BD%D0%B5%D1%80%D0%B0%D1%86%D0%B8%D0%B8_%D0%B2%D1%8B%D1%81%D0%BE%D0%BA%D0%BE%D0%BA%D0%B0%D1%87%D0%B5%D1%81%D1%82%D0%B2%D0%B5%D0%BD%D0%BD%D1%8B%D1%85_%D0%B8%D0%B7%D0%BE%D0%B1%D1%80%D0%B0%D0%B6%D0%B5%D0%BD%D0%B8%D0%B9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Установка необходимых библиотек
!pip install transformers diffusers accelerate torch torchvision matplotlib pytorch-fid datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.3 MB/s eta 0:00:00


In [3]:
# Импорт необходимых библиотек
import torch
from diffusers import StableDiffusionPipeline, StableDiffusionInpaintPipeline, DDPMScheduler, UNet2DModel
from transformers import CLIPTextModel, CLIPTokenizer
from datasets import load_dataset
from PIL import Image
import matplotlib.pyplot as plt
from pytorch_fid import fid_score
import os

# Проверка доступности GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# 1. Анализ архитектуры модели
def analyze_model_architecture():
    print("Analyzing model architecture...")
    model_id = "CompVis/stable-diffusion-v1-4"
    pipe = StableDiffusionPipeline.from_pretrained(model_id, revision="fp16", torch_dtype=torch.float16)

    # Вывод информации о компонентах модели
    print("\nU-Net Architecture:")
    print(pipe.unet)

    print("\nVAE (Variational Autoencoder) Architecture:")
    print(pipe.vae)

    print("\nText Encoder Architecture:")
    print(pipe.text_encoder)

# 2. Оптимизация параметров генерации
def optimize_generation_parameters(prompt):
    guidance_scales = [5.0, 7.5, 10.0]
    num_inference_steps = [25, 50, 100]

    for scale in guidance_scales:
        for steps in num_inference_steps:
            print(f"\nGenerating images with guidance_scale={scale}, num_inference_steps={steps}")
            images = generate_image(prompt, guidance_scale=scale, num_inference_steps=steps)
            display_images(images)

# Функция для генерации изображения
def generate_image(prompt, num_images=1, guidance_scale=7.5, num_inference_steps=50):
    model_id = "CompVis/stable-diffusion-v1-4"
    pipe = StableDiffusionPipeline.from_pretrained(model_id, revision="fp16", torch_dtype=torch.float16).to(device)
    with torch.autocast("cuda"):
        images = pipe(
            prompt,
            num_images_per_prompt=num_images,
            guidance_scale=guidance_scale,
            num_inference_steps=num_inference_steps
        ).images
    return images

# Функция для отображения изображений
def display_images(images):
    plt.figure(figsize=(15, 15))
    for i, image in enumerate(images):
        plt.subplot(1, len(images), i + 1)
        plt.imshow(image)
        plt.axis('off')
    plt.show()

# 3. Сравнение разных моделей
def compare_models(prompt):
    model_ids = ["CompVis/stable-diffusion-v1-4", "stabilityai/stable-diffusion-2-base"]
    for model_id in model_ids:
        print(f"\nGenerating images using model: {model_id}")
        pipe = StableDiffusionPipeline.from_pretrained(model_id, revision="fp16", torch_dtype=torch.float16).to(device)
        images = generate_image(prompt)
        display_images(images)

# 4. Анализ качества генерации (FID)
def analyze_quality():
    real_images_path = "path_to_real_images"  # Замените на путь к реальным изображениям
    generated_images_path = "path_to_generated_images"  # Замените на путь к сгенерированным изображениям

    if not os.path.exists(real_images_path) or not os.path.exists(generated_images_path):
        print("Paths to real and generated images are invalid.")
        return

    fid_value = fid_score.calculate_fid_given_paths(
        [real_images_path, generated_images_path],
        batch_size=50,
        device=device,
        dims=2048
    )
    print(f"FID score: {fid_value}")

# 5. Изучение ограничений модели
def study_model_limitations():
    print("Studying model limitations...")
    # Пример сложного текстового запроса
    complex_prompt = "A realistic photo of a flying dragon with detailed scales and fire breathing"
    images = generate_image(complex_prompt)
    display_images(images)
    print("Note: The model may struggle with generating highly specific or rare objects.")

# 6. Расширение возможностей (Генерация с маской)
def extend_capabilities():
    print("Extending capabilities with inpainting...")
    pipe_inpaint = StableDiffusionInpaintPipeline.from_pretrained("runwayml/stable-diffusion-inpainting").to(device)

    # Загрузка начального изображения и маски
    input_image = Image.open("input_image.jpg").convert("RGB")
    mask_image = Image.open("mask_image.png").convert("RGB")

    # Генерация изображения с учетом маски
    output = pipe_inpaint(prompt="A beautiful landscape", image=input_image, mask_image=mask_image).images[0]
    output.save("inpaint_output.png")
    display_images([output])

# 7. Обучение модели (опционально)
def train_custom_model():
    print("Training a custom diffusion model...")
    dataset = load_dataset("huggan/smithsonian_butterflies_subset")

    # Создание модели
    model = UNet2DModel(
        sample_size=32,  # Размер входных данных
        in_channels=3,   # RGB
        out_channels=3,
        layers_per_block=2,
        block_out_channels=(128, 128, 256, 256, 512, 512),
        down_block_types=(
            "DownBlock2D",
            "DownBlock2D",
            "DownBlock2D",
            "DownBlock2D",
            "AttnDownBlock2D",
            "AttnDownBlock2D"
        ),
        up_block_types=(
            "AttnUpBlock2D",
            "AttnUpBlock2D",
            "UpBlock2D",
            "UpBlock2D",
            "UpBlock2D",
            "UpBlock2D"
        )
    )

    # Настройка планировщика шума
    noise_scheduler = DDPMScheduler(num_train_timesteps=1000)

    # Простой пример обучения (требует дополнительной реализации)
    print("Training setup complete. Actual training requires more code.")

# Основная функция
if __name__ == "__main__":
    # Текстовый запрос для генерации
    prompt = "A beautiful sunset over the ocean with a lighthouse in the distance"

    # 1. Анализ архитектуры модели
    analyze_model_architecture()

    # 2. Оптимизация параметров генерации
    optimize_generation_parameters(prompt)

    # 3. Сравнение разных моделей
    compare_models(prompt)

    # 4. Анализ качества генерации
    analyze_quality()

    # 5. Изучение ограничений модели
    study_model_limitations()

    # 6. Расширение возможностей
    extend_capabilities()

    # 7. Обучение модели (опционально)
    train_custom_model()



Using device: cpu
Analyzing model architecture...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/543 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/diffusers/pipelines/pipeline_loading_utils.py:242: FutureWarning: You are loading the variant fp16 from CompVis/stable-diffusion-v1-4 via `revision='fp16'` even though you can load it via `variant=`fp16`. Loading model variants via `revision='fp16'` is deprecated and will be removed in diffusers v1. Please use `variant='fp16'` instead.
  warnings.warn(


Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

...


U-Net Architecture:
UNet2DConditionModel(
  (conv_in): Conv2d(4, 320, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (time_proj): Timesteps()
  (time_embedding): TimestepEmbedding(
    (linear_1): Linear(in_features=320, out_features=1280, bias=True)
    (act): SiLU()
    (linear_2): Linear(in_features=1280, out_features=1280, bias=True)
  )
  (down_blocks): ModuleList(
    (0): CrossAttnDownBlock2D(
      (attentions): ModuleList(
        (0-1): 2 x Transformer2DModel(
          (norm): GroupNorm(32, 320, eps=1e-06, affine=True)
          (proj_in): Conv2d(320, 320, kernel_size=(1, 1), stride=(1, 1))
          (transformer_blocks): ModuleList(
            (0): BasicTransformerBlock(
              (norm1): LayerNorm((320,), eps=1e-05, elementwise_affine=True)
              (attn1): Attention(
                (to_q): Linear(in_features=320, out_features=320, bias=False)
                (to_k): Linear(in_features=320, out_features=320, bias=False)
                (to_v): Linear(in

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

An error occurred while trying to fetch /root/.cache/huggingface/hub/models--CompVis--stable-diffusion-v1-4/snapshots/2880f2ca379f41b0226444936bb7a6766a227587/unet: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--CompVis--stable-diffusion-v1-4/snapshots/2880f2ca379f41b0226444936bb7a6766a227587/unet.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
An error occurred while trying to fetch /root/.cache/huggingface/hub/models--CompVis--stable-diffusion-v1-4/snapshots/2880f2ca379f41b0226444936bb7a6766a227587/vae: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--CompVis--stable-diffusion-v1-4/snapshots/2880f2ca379f41b0226444936bb7a6766a227587/vae.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
Pipelines loaded with `dtype=torch.float16` cannot run with `cpu` device. It is not recommende

  0%|          | 0/25 [00:00<?, ?it/s]

KeyboardInterrupt: 